# Backfill and Feature Engineering Notebook
This notebook consists of 5 parts:
1. Importing libaries and loading packages
2. Data Loading
3. Data Preprocessing
4. Feature Engineering
5. Hopsworks Feature Storage

Throughout this notebook, our decision-making process is informed by insights gained from the exploratory data analysis (EDA) we conducted. This analysis helped us identify the most relevant information for our methods and strategies

## 1. Importing libaries and loading packages
In this section, we import necessary libraries and define key functions.

In [1]:
# Package for hopsworks integration
# !pip install -U hopsworks --quiet

# Import standard Python libraries
import pandas as pd 
import hopsworks 
import numpy as np 

# Import machine learning tools
from sklearn.preprocessing import StandardScaler  
from sklearn.cluster import KMeans  
from sklearn.metrics import silhouette_score  

# Import other useful libraries
import uuid  # Unique identifier generation
import requests  # For making API requests
import json  
import io 
import base64 
from datetime import datetime, timedelta  # Date/time handling and manipulation
import pytz  # Timezone conversions and support

## 2. Data Loading
We load historic data to be used in the notebook.

- The data is gathered trough an API given by the company, and the API data results are stored in the *bikelane_historic_data.csv* and *building_historic_data.csv*. The data loacated in the *data* folder is old data that the EDA and end-to-end-pipeline is made from. Here you can also find more information about the data in the readme.txt file.

- The data derivied from the API is from two parking spots, one of the spots are close to a building and the other close to a bikelane, and will be refered to with this as the identifyer.

- In this section we will be pinging the API and saving historic data from march and april in two csv-files, containing data from the parking spot close to the building and the one close to the bikelane

In [2]:
# Create a timezone object for GMT+2
timezone = pytz.timezone('Europe/Bucharest')
now = datetime.now(pytz.utc)  # Get current time in UTC
today = now.astimezone(timezone)
yesterday = today - timedelta(days=1)
tomorrow = today + timedelta(days=1)

In [3]:
#Today', 'tomorrow', and 'yesterday' as "YYYY-MM-DD"
#Today = today.strftime('%Y-%m-%d')
#Tomorrow = tomorrow.strftime('%Y-%m-%d')
#Yesterday = yesterday.strftime('%Y-%m-%d')

url = "https://data.sensade.com"
dev_eui_building = "0080E115003BEA91"
dev_eui_bikelane = "0080E115003E3597"
username = "ajakup20@student.aau.dk"
pwd = "API_PASSWORD"
basic_auth = base64.b64encode(f"{username}:{pwd}".encode())
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Basic {basic_auth.decode("utf-8")}'
}

In [4]:
# Function to ping the API and get data in a given time interval
def API_call(dev_eui, from_date, to_date):
    payload = json.dumps({
    "dev_eui": dev_eui,
    "from": from_date,
    "to": to_date
})

    API_response = requests.request("GET", url, headers=headers, data=payload)

    if API_response.status_code != 200:
        exit(13)

    csv_data = API_response.text
    df = pd.read_csv(io.StringIO(csv_data))
    return df

This section below is commented out because the files are already created

In [5]:
# march_1_building = API_call(dev_eui_building, "2024-03-01", "2024-03-14")
# march_2_building = API_call(dev_eui_building, "2024-03-15", "2024-03-31")
# april_1_building = API_call(dev_eui_building, "2024-04-01", "2024-04-14")
# april_2_building = API_call(dev_eui_building, "2024-04-15", "2024-04-30")# 

# march_1_bikelane = API_call(dev_eui_bikelane, "2024-03-01", "2024-03-14")
# march_2_bikelane = API_call(dev_eui_bikelane, "2024-03-15", "2024-03-31")
# april_1_bikelane = API_call(dev_eui_bikelane, "2024-04-01", "2024-04-14")
# april_2_bikelane = API_call(dev_eui_bikelane, "2024-04-15", "2024-04-30")

# building_historic_df = pd.concat([march_1_building, march_2_building, april_1_building, april_2_building], ignore_index=True)
# bikelane_historic_df = pd.concat([march_1_bikelane, march_2_bikelane, april_1_bikelane, april_2_bikelane], ignore_index=True)

# saving the data as CSV
# building_historic_df.to_csv('building_historic_data.csv', index=False)
# bikelane_historic_df.to_csv('bikelane_historic_data.csv', index=False)

In [5]:
#loading the data saved in the directory
building_historic_df = pd.read_csv('building_historic_data.csv')
bikelane_historic_df = pd.read_csv('bikelane_historic_data.csv')

## 3. Data Preprocessing
Now before just dumping the data into a feature store we do a little preprocessing to enhance the use of our datasets.

This preprocessing consists of:
- Making unique identifyers for each datapoint
- Combining the three datasets into one 
- Making clusters used for labeling, which is nessesary when we want to train our models later
- Converting the data column to pandas datetime
- minor adjustments for the naming of radar columns to fix some hopsworks problem where the name of the columns cannot start with a number, and making the relevant columns into float format

In [6]:
# Create a unique identifier for each row in the datasets
def create_id(df, dataset_name):
    # Assign the sensor prefix based on the dataset name
    if dataset_name == 'df_building_historic':
        df['psensor'] = "BUILDING"
    elif dataset_name == 'df_bikelane_historic':
        df['psensor'] = "BIKELANE"
    else:
        raise ValueError("Unknown dataset name provided")

    # Create a new column 'id' with a unique identifier for each row
    df['id'] = [str(uuid.uuid4()) for _ in df.index]

    return df

In [7]:
# Applying the ID creator function to the datasets
bikelane_historic_df = bikelane_historic_df.copy()
df_bikelane = create_id(bikelane_historic_df, 'df_bikelane_historic')
df_building_historic = building_historic_df.copy()
df_building = create_id(df_building_historic, 'df_building_historic')

In [8]:
# Convert the 'time' column to datetime, coercing errors to NaT
df_building['time'] = pd.to_datetime(df_building['time'], errors='coerce')
df_bikelane['time'] = pd.to_datetime(df_bikelane['time'], errors='coerce')
# Remove rows where the 'time' column is NaT
df_building = df_building.dropna(subset=['time'])
df_bikelane = df_bikelane.dropna(subset=['time'])

In [9]:
#Renaming the radar columns to start with radar to deal with hopsworks problem
df_building = df_building.rename(columns={'0_radar': 'radar_0', '1_radar': 'radar_1', '2_radar': 'radar_2', '3_radar': 'radar_3', '4_radar': 'radar_4', '5_radar': 'radar_5', '6_radar': 'radar_6', '7_radar': 'radar_7'})
df_bikelane = df_bikelane.rename(columns={'0_radar': 'radar_0', '1_radar': 'radar_1', '2_radar': 'radar_2', '3_radar': 'radar_3', '4_radar': 'radar_4', '5_radar': 'radar_5', '6_radar': 'radar_6', '7_radar': 'radar_7'})


In [10]:
# Converting the columns to float
df_building[['x','y','z', 'radar_0', 'radar_1', 'radar_2', 'radar_3', 'radar_4', 'radar_5', 'radar_6', 'radar_7', 'f_cnt', 'dr', 'rssi']] = df_building[['x','y','z', 'radar_0', 'radar_1', 'radar_2', 'radar_3', 'radar_4', 'radar_5', 'radar_6', 'radar_7', 'f_cnt', 'dr', 'rssi']].astype(float)
df_bikelane[['x','y','z', 'radar_0', 'radar_1', 'radar_2', 'radar_3', 'radar_4', 'radar_5', 'radar_6', 'radar_7', 'f_cnt', 'dr', 'rssi']] = df_bikelane[['x','y','z', 'radar_0', 'radar_1', 'radar_2', 'radar_3', 'radar_4', 'radar_5', 'radar_6', 'radar_7', 'f_cnt', 'dr', 'rssi']].astype(float)


## 4. Feature Engineering
In this step, we develop a method to label the data points as either 'detection' or 'no_detection.' 

Our exploratory data analysis revealed that the electromagnetic field data is best suited for our objectives. Therefore, we focus on the x, y, and z data from this dataset.

In our case, we chose KMeans as our clustering method and used the magnetic sensor data from the x, y, and z axes as features. This is done after normalizing the data using StandardScaler.

In [11]:
# Making a dataframe for the features we wish to cluster on
building_mag = df_building[["x","y","z"]]
bikelane_mag = df_bikelane[["x","y","z"]]

In [12]:
# Normalizing the data
scaler = StandardScaler()
building_mag_norm = scaler.fit_transform(building_mag)
bikelane_mag_norm = scaler.fit_transform(bikelane_mag)
# Clustering the magnetic field data with 2 clusters using kmeans
building_kmeans = KMeans(n_clusters=2, random_state=0).fit(building_mag_norm)
bikelane_kmeans = KMeans(n_clusters=2, random_state=0).fit(bikelane_mag_norm)


In [13]:
# Adding cluster labels to the mag dataframe
building_mag = building_mag.copy() #dealing with an error
bikelane_mag = bikelane_mag.copy() #dealing with an error
building_mag['mag_cluster'] = building_kmeans.labels_
bikelane_mag['mag_cluster'] = bikelane_kmeans.labels_
df_building = df_building.copy() #dealing with an error
df_bikelane = df_bikelane.copy() #dealing with an error
df_building['mag_cluster'] = building_mag['mag_cluster']
df_bikelane['mag_cluster'] = bikelane_mag['mag_cluster']

In [14]:
# Renaming the cluster labels to 'detection' and 'no_detection'
df_building['mag_cluster'].replace({0: 'no_detection', 1: 'detection'}, inplace=True)
df_bikelane['mag_cluster'].replace({0: 'no_detection', 1: 'detection'}, inplace=True)

In [15]:
# Fixing an error with the mag_cluster column type
df_building['mag_cluster'] = df_building['mag_cluster'].astype(str)
df_building['mag_cluster'].replace('nan', None, inplace=True)  # Replace 'nan' string with actual None
df_bikelane['mag_cluster'] = df_bikelane['mag_cluster'].astype(str)
df_bikelane['mag_cluster'].replace('nan', None, inplace=True)  # Replace 'nan' string with actual None

## 5. Hopsworks Feature Storage

Now we would like to connect to the Hopsworks Feature Store so we can access and create feature groups.

In creating feature groups we take all the relevant coulmns and store it in hopworks, so that we later can acces and interperet for further use.

We also specify a 'primary_key' that is used for relating diferent dimention tables to each other, in our case this is the unique ID that we made in the preprocessing step. 

The 'time' column is used as the event time key.

we also we put `online_enabled` to `True` to make the feature group online for acces with an API when we make feature views.

And finally we give descriptions to each coulmn with information given by the *README.txt* in *data*. 


In [16]:
# Connceting to the Hopsworks project
project = hopsworks.login()
fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated
Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) annikaij
	 (2) miknie20

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549019
Connected. Call `.close()` to terminate connection gracefully.


In [17]:
# Create a feature group for the parking spot close to the building
api_hist_building_fg = fs.get_or_create_feature_group(
    name="api_building_detection_features",
    version=1,
    description="Data from API for parking detection on the parkingspot close to the building",
    primary_key=['id'],
    event_time='time',
    online_enabled=True
)

In [18]:
# Insert the magnetic field features into the feature group
api_hist_building_fg.insert(df_building, write_options={"wait_for_job" : False})


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/549019/fs/544841/fg/794956


Uploading Dataframe: 0.00% |          | Rows 0/7155 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: api_building_detection_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549019/jobs/named/api_building_detection_features_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f936758ae90>, None)

In [19]:
# Making descriptions for the features
feature_descriptions = [
    {"name": "time", "description": "Timepoint of the datapoint"},
    {"name": "battery", "description": "Battery level of the sensor"},
    {"name": "temperature", "description": "Temperature recorded by the sensor"},
    {"name": "x", "description": "Magnetic field reading in the x direction"},
    {"name": "y", "description": "Magnetic field reading in the y direction"},
    {"name": "z", "description": "Magnetic field reading in the z direction"},
    {"name": "radar_0", "description": "Radar reading from sensor radar sensor 0"},
    {"name": "radar_1", "description": "Radar reading from sensor radar sensor 1"},
    {"name": "radar_2", "description": "Radar reading from sensor radar sensor 2"},
    {"name": "radar_3", "description": "Radar reading from sensor radar sensor 3"},
    {"name": "radar_4", "description": "Radar reading from sensor radar sensor 4"},
    {"name": "radar_5", "description": "Radar reading from sensor radar sensor 5"},
    {"name": "radar_6", "description": "Radar reading from sensor radar sensor 6"},
    {"name": "radar_7", "description": "Radar reading from sensor radar sensor 7"},
    {"name": "package_type", "description": "Heartbeat indicates no significant change since last reading or change package type means that x, y or z has changed significantly +-30"},
    {"name": "f_cnt", "description": "number of packages transmitted since last network registration"},
    {"name": "dr", "description": "data rate parameter in LoRaWAN. It ranges between 1 and 5 where 1 is the slowest transmission data rate and 5 is the highest. This datarate is scaled by the network server depending on the signal quality of the past packages send"},
    {"name": "snr", "description": "signal to noise ratio – the higher value, the better the signal quality"},
    {"name": "rssi", "description": "signal strength – the higher value, the better the signal quality"},
    {"name": "psensor", "description": "sensor identifier (ex. EL1, EL2, EL3)"},
    {"name": "hw_fw_version", "description": "hardware and firmware version of the sensor"},
    {"name": "id", "description": "unique identifier for each datapoint made uuid4"}
]

for desc in feature_descriptions: 
    api_hist_building_fg.update_feature_description(desc["name"], desc["description"])

In [20]:
# Create a feature group for the parking spot close to the bike lane
api_hist_bikelane_fg = fs.get_or_create_feature_group(
    name="api_bikelane_detection_features",
    version=1,
    description="Data from API for parking detection on the parkingspot close to the bikelane",
    primary_key=['id'],
    event_time='time',
    online_enabled=True
)

In [21]:
# Insert the magnetic field features into the feature group
api_hist_bikelane_fg.insert(df_bikelane, write_options={"wait_for_job" : False})


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/549019/fs/544841/fg/793932


Uploading Dataframe: 0.00% |          | Rows 0/7047 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: api_bikelane_detection_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549019/jobs/named/api_bikelane_detection_features_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f936755ff50>, None)

In [22]:
# Making descriptions for the features
feature_descriptions = [
    {"name": "time", "description": "Timepoint of the datapoint"},
    {"name": "battery", "description": "Battery level of the sensor"},
    {"name": "temperature", "description": "Temperature recorded by the sensor"},
    {"name": "x", "description": "Magnetic field reading in the x direction"},
    {"name": "y", "description": "Magnetic field reading in the y direction"},
    {"name": "z", "description": "Magnetic field reading in the z direction"},
    {"name": "radar_0", "description": "Radar reading from sensor radar sensor 0"},
    {"name": "radar_1", "description": "Radar reading from sensor radar sensor 1"},
    {"name": "radar_2", "description": "Radar reading from sensor radar sensor 2"},
    {"name": "radar_3", "description": "Radar reading from sensor radar sensor 3"},
    {"name": "radar_4", "description": "Radar reading from sensor radar sensor 4"},
    {"name": "radar_5", "description": "Radar reading from sensor radar sensor 5"},
    {"name": "radar_6", "description": "Radar reading from sensor radar sensor 6"},
    {"name": "radar_7", "description": "Radar reading from sensor radar sensor 7"},
    {"name": "package_type", "description": "Heartbeat indicates no significant change since last reading or change package type means that x, y or z has changed significantly +-30"},
    {"name": "f_cnt", "description": "number of packages transmitted since last network registration"},
    {"name": "dr", "description": "data rate parameter in LoRaWAN. It ranges between 1 and 5 where 1 is the slowest transmission data rate and 5 is the highest. This datarate is scaled by the network server depending on the signal quality of the past packages send"},
    {"name": "snr", "description": "signal to noise ratio – the higher value, the better the signal quality"},
    {"name": "rssi", "description": "signal strength – the higher value, the better the signal quality"},
    {"name": "psensor", "description": "sensor identifier (ex. EL1, EL2, EL3)"},
    {"name": "hw_fw_version", "description": "hardware and firmware version of the sensor"},
    {"name": "id", "description": "unique identifier for each datapoint made uuid4"}
]

for desc in feature_descriptions: 
    api_hist_bikelane_fg.update_feature_description(desc["name"], desc["description"])

## **Next up:** 2: Latest API data
Go to the 2_latest_api_feature_pipeline.ipynb notebook